In [2]:
import json
from pathlib import Path

In [3]:
# Path to users.json
users_path = Path("../context/users/users.json")

In [6]:
def load_users():
    if users_path.exists():
        with open(users_path, "r") as f:
            return json.load(f)
    return []

In [7]:
def get_user_by_id(user_id, users):
    for user in users:
        if user.get("user_id") == user_id:
            return user
    return None

In [11]:
get_user_by_id('00006', load_users())

{'user_id': '00006',
 'created_at': '2025-06-11T20:53:48.928579',
 'persona': 'spiritual_explorer',
 'view': 'christian',
 'step': None,
 'step_in_progress': False}

In [8]:
load_users()

[{'user_id': '00001',
  'created_at': '2025-06-01T12:00:00Z',
  'persona': 'steady_believer',
  'view': 'christian',
  'step': 2,
  'step_in_progress': True},
 {'user_id': '00002',
  'created_at': '2025-06-01T12:05:00Z',
  'persona': 'trust_builder',
  'view': 'atheist',
  'step': 2,
  'step_in_progress': True},
 {'user_id': '00003',
  'created_at': '2025-06-01T12:10:00Z',
  'persona': 'gentle_seeker',
  'view': 'agnostic',
  'step': 2,
  'step_in_progress': True},
 {'user_id': '00004',
  'created_at': '2025-06-11T19:15:38.210758',
  'persona': 'spiritual_explorer',
  'view': 'unknown',
  'step': None,
  'step_in_progress': False},
 {'user_id': '00005',
  'created_at': '2025-06-11T20:37:48.761045',
  'persona': 'spiritual_explorer',
  'view': 'christian',
  'step': None,
  'step_in_progress': False},
 {'user_id': '00006',
  'created_at': '2025-06-11T20:53:48.928579',
  'persona': 'spiritual_explorer',
  'view': 'christian',
  'step': None,
  'step_in_progress': False}]

In [ ]:
# Input user_id and load user info
user_id = input("Enter your user_id: ").strip()
users = load_users()
user = get_user_by_id(user_id, users)

if user:
    print(f"User found: {user}")
else:
    print("User not found.")